In [ ]:
# import sys
# from pathlib import Path

# # Get the absolute path to the project root
# notebook_path = Path('/Users/nirmal/Documents/np_research/ManGo_code/ManGo')
# project_root = str(notebook_path)

# # Add the project root to Python path if it's not already there
# if project_root not in sys.path:
#     sys.path.insert(0, project_root)

# # Now you can import your module
# from src.models.mg_rig_wrp import GnosticRobustRegressor

# # Verify the import worked
# # print(f"Project root: {project_root}")
# # print(f"Available at: {RobustRegressor.__module__}")

In [4]:
import pandas as pd
import mlflow.pyfunc
from mlflow.models.signature import infer_signature
from machinegnostics.models import RobustRegressor
import numpy as np
# Create and train the model
model = RobustRegressor(degree=3, mg_loss='hi', verbose=True)
X = np.random.rand(100, 2)
y = np.random.rand(100)
model.fit(X, y)

# Create an input example
input_example = pd.DataFrame(X, columns=["feature1", "feature2"])

# Infer the model signature
signature = infer_signature(input_example, model.predict( input_example))

# Log the model to MLflow with input_example and signature
with mlflow.start_run():
    mlflow.pyfunc.log_model(
        artifact_path="robust_regressor",
        python_model=model,
        registered_model_name="GnosticRobustRegressor",
        input_example=input_example,
        signature=signature
    )

2025/05/26 08:30:06 INFO mlflow.pyfunc: Validating input example against model signature


Iteration: 1 - Machine Gnostic loss - hi : 81.2747, rentropy: 58.8221
Iteration: 2 - Machine Gnostic loss - hi : 84.8199, rentropy: 107.3571
Iteration: 3 - Machine Gnostic loss - hi : 84.8372, rentropy: 171.4301
Iteration: 4 - Machine Gnostic loss - hi : 84.3212, rentropy: 205.1403
Iteration: 5 - Machine Gnostic loss - hi : 83.8684, rentropy: 210.6295
Iteration: 6 - Machine Gnostic loss - hi : 83.5615, rentropy: 206.1767
Iteration: 7 - Machine Gnostic loss - hi : 83.3763, rentropy: 200.341
Iteration: 8 - Machine Gnostic loss - hi : 83.2718, rentropy: 194.8361
Iteration: 9 - Machine Gnostic loss - hi : 83.2165, rentropy: 189.662
Iteration: 10 - Machine Gnostic loss - hi : 83.1901, rentropy: 185.3468
Iteration: 11 - Machine Gnostic loss - hi : 83.18, rentropy: 181.9664
Iteration: 12 - Machine Gnostic loss - hi : 83.1786, rentropy: 179.5406
Iteration: 13 - Machine Gnostic loss - hi : 83.1808, rentropy: 177.7294
Iteration: 14 - Machine Gnostic loss - hi : 83.1845, rentropy: 176.5147
Iterat

2025/05/26 08:30:08 WARNING mlflow.utils.requirements_utils: The following packages were not found in the public PyPI package index as of 2025-04-15; if these packages are not present in the public PyPI index, you must install them manually before loading your model: {'machinegnostics'}
Registered model 'GnosticRobustRegressor' already exists. Creating a new version of this model...
Created version '10' of model 'GnosticRobustRegressor'.


In [6]:
# Load the model from MLflow
loaded_model = mlflow.pyfunc.load_model("models:/GnosticRobustRegressor/1")

# Use the model for predictions
import pandas as pd
X_test = pd.DataFrame(np.random.rand(10, 2), columns=["feature1", "feature2"])
predictions = loaded_model.predict(X_test)
print(predictions)

[0.72860525 1.01340748 0.66627017 0.42181381 0.55867998 0.70712085
 0.5633641  0.41021955 0.63907862 0.48802059]


In [1]:
import numpy as np
import pandas as pd
import mlflow
import mlflow.pyfunc
from mlflow.models.signature import infer_signature
from machinegnostics.models import RobustRegressor  # Replace with your actual import path

# Step 1: Set up MLflow experiment
mlflow.set_experiment("Robust Regressor Experiment")

# Step 2: Generate synthetic data
X = np.random.rand(100, 2)  # 100 samples, 2 features
y = 2.5 * X[:, 0] + 1.5 * X[:, 1] + np.random.normal(scale=0.1, size=100)  # Linear relationship with noise

# Step 3: Train the model
model = RobustRegressor(degree=3, mg_loss='hi', verbose=True)
model.fit(X, y)

# Step 4: Create an input example and infer the model signature
input_example = pd.DataFrame(X, columns=["feature1", "feature2"])
signature = infer_signature(model_input=input_example, model_output=y)

# Step 5: Log the model, parameters, and metrics to MLflow
with mlflow.start_run():
    # Log parameters
    mlflow.log_param("degree", 3)
    mlflow.log_param("mg_loss", "hi")
    mlflow.log_param("verbose", True)

    # Log metrics (example: training RMSE)
    y_pred = model.predict(input_example)
    rmse = np.sqrt(np.mean((y - y_pred) ** 2))
    mlflow.log_metric("train_rmse", rmse)

    # Log the model
    mlflow.pyfunc.log_model(
        artifact_path="robust_regressor",
        python_model=model,
        registered_model_name="GnosticRobustRegressor",
        input_example=input_example,
        signature=signature
    )



# Step 8: Serve the model as a REST API (optional)
# Run this command in the terminal to serve the model:
# mlflow models serve -m "models:/GnosticRobustRegressor/1" --port 5000
# Then, make predictions via REST API:
# curl -X POST -H "Content-Type: application/json" --data '{"columns":["feature1", "feature2"], "data":[[0.1, 0.2], [0.3, 0.4]]}' http://127.0.0.1:5000/invocations

/Users/nirmal/Documents/np_research/ManGo_code/ManGo/.venv/lib/python3.11/site-packages/mlflow/pyfunc/utils/data_validation.py:186: UserWarning: Add type hints to the `predict` method to enable data validation and automatic signature inference during model logging. Check https://mlflow.org/docs/latest/model/python_model.html#type-hint-usage-in-pythonmodel for more details.
  color_warning(
/Users/nirmal/Documents/np_research/ManGo_code/ManGo/.venv/lib/python3.11/site-packages/mlflow/pyfunc/utils/data_validation.py:186: UserWarning: Add type hints to the `predict` method to enable data validation and automatic signature inference during model logging. Check https://mlflow.org/docs/latest/model/python_model.html#type-hint-usage-in-pythonmodel for more details.
  color_warning(
/Users/nirmal/Documents/np_research/ManGo_code/ManGo/.venv/lib/python3.11/site-packages/mlflow/pyfunc/utils/data_validation.py:186: UserWarning: Add type hints to the `predict` method to enable data validation and 

Iteration: 1 - Machine Gnostic loss - hi : 82.2087, rentropy: 277.0078
Iteration: 2 - Machine Gnostic loss - hi : 80.8069, rentropy: 74.4022
Iteration: 3 - Machine Gnostic loss - hi : 82.3267, rentropy: 79.2043
Iteration: 4 - Machine Gnostic loss - hi : 83.3677, rentropy: 84.4172
Iteration: 5 - Machine Gnostic loss - hi : 84.0683, rentropy: 87.3153
Iteration: 6 - Machine Gnostic loss - hi : 84.5589, rentropy: 89.5891
Iteration: 7 - Machine Gnostic loss - hi : 84.8238, rentropy: 91.7296
Iteration: 8 - Machine Gnostic loss - hi : 84.9197, rentropy: 94.0374
Iteration: 9 - Machine Gnostic loss - hi : 84.9247, rentropy: 96.8907
Iteration: 10 - Machine Gnostic loss - hi : 84.8535, rentropy: 101.1386
Iteration: 11 - Machine Gnostic loss - hi : 84.6262, rentropy: 109.3796
Iteration: 12 - Machine Gnostic loss - hi : 84.3143, rentropy: 131.6897
Iteration: 13 - Machine Gnostic loss - hi : 83.654, rentropy: 152.9742
Iteration: 14 - Machine Gnostic loss - hi : 82.8582, rentropy: 147.7802
Iteration:

2025/05/26 08:29:11 WARNING mlflow.utils.requirements_utils: The following packages were not found in the public PyPI package index as of 2025-04-15; if these packages are not present in the public PyPI index, you must install them manually before loading your model: {'machinegnostics'}
Registered model 'GnosticRobustRegressor' already exists. Creating a new version of this model...
Created version '9' of model 'GnosticRobustRegressor'.


In [7]:
# Step 6: Load the model from MLflow
loaded_model = mlflow.pyfunc.load_model("models:/GnosticRobustRegressor/9")

# Step 7: Use the loaded model for predictions
X_test = pd.DataFrame(np.random.rand(10, 2), columns=["feature1", "feature2"])
predictions = loaded_model.predict(X_test)
print("Predictions:")
print(predictions)

Predictions:
[1.66574    1.25060256 3.17256284 1.90769642 1.43856593 1.13476993
 2.78816187 2.67159081 2.23867922 1.61363489]
